In [21]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

In [22]:
import warnings 
warnings.filterwarnings('ignore')
lookback_window = [0, 50]
number_of_clusters = 38

def cluster_composition_and_centroid(self):
## cluster composition and centroids

result = dict(zip(list(self.correlation_matrix.columns), self.apply_SPONGE())) ## composition

df_cleaned['Cluster'] = df_cleaned.index.map(result)
centroid_returns = df_cleaned.groupby('Cluster').mean() ## centroids 

df_cleaned = df_cleaned.transpose()
centroid_returns = centroid_returns.transpose()

## constituent_weights
centred_returns = df_cleaned.copy()
sigma = 0.01

constituent_weights = pd.DataFrame(index=['Weight'], columns=centred_returns.columns)
total_weight = pd.DataFrame(index=['Total weight'], columns=[i for i in range(number_of_clusters)], data=np.zeros((1, number_of_clusters)))

## we first compute the difference between the cluster centroid return and the cluster ticker return
for ticker in centred_returns.columns:
    centred_returns[ticker][:-1] = centred_returns[ticker][:-1] - centroid_returns[int(centred_returns[ticker]['Cluster'])]

## we use this difference to compute the distance between each asset and its cluster centroid return 
for ticker in centred_returns.columns:
    constituent_weights[ticker] = np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)
    total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight'] += np.exp(sigma*((np.linalg.norm(centred_returns[ticker][:-1]))**2)/2)

## we normalize the weights
for ticker in centred_returns.columns:
    constituent_weights[ticker] = constituent_weights[ticker]['Weight']/total_weight[int(centred_returns[ticker]['Cluster'])]['Total weight']

## check whether the weights equal to 0 within each cluster: 
# constituent_weights[[ticker for ticker in centred_returns.columns if centred_returns[ticker]['Cluster']  == 1.0]].sum(axis=1)

In [74]:
df_cleaned

ticker,AA,ABM,ABT,ADI,ADM,ADX,AEE,AEG,AEM,AEP,...,XLI,XLK,XLP,XLU,XLV,XLY,XOM,XRX,YUM,ZTR
03/01/2000,-0.013042,-0.009188,-0.007117,-0.036071,0.000000,-0.001867,-0.007707,0.014739,-0.008658,-0.017734,...,-0.021967,-0.003376,-0.016349,-0.017490,-0.022942,-0.028632,-0.016647,0.034169,-0.032952,0.000000
04/01/2000,0.010043,0.012346,-0.012786,-0.044261,0.005277,-0.005666,0.000000,-0.018361,0.018019,0.013848,...,-0.014930,-0.025793,-0.009145,-0.031198,-0.019337,-0.015915,-0.007296,-0.047628,-0.011895,0.018868
05/01/2000,0.047628,-0.006192,0.011111,0.014493,-0.015915,0.000000,0.037955,-0.012089,-0.017858,0.036648,...,-0.002794,0.001810,0.010471,0.002866,-0.014799,-0.016645,0.037740,0.021506,0.000000,0.009479
06/01/2000,-0.011713,0.000000,0.032553,-0.027719,0.010695,0.005742,-0.003731,-0.003409,0.026907,0.013270,...,0.013341,-0.022196,0.011656,0.000000,0.004255,0.004829,0.052842,0.007926,-0.001714,0.009390
07/01/2000,-0.016118,0.003091,0.028573,0.033654,0.005249,0.003810,0.012975,0.000667,0.026202,0.007463,...,0.025864,0.027331,0.043898,0.006845,0.011073,0.035384,-0.011705,0.023408,-0.022551,0.009217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28/12/2020,0.006315,0.009932,-0.010796,-0.011887,-0.002010,0.004078,0.001565,-0.005128,-0.027193,-0.005643,...,-0.005320,0.001380,0.004172,-0.000813,-0.002681,0.001630,0.001199,0.020396,0.011820,0.005705
29/12/2020,-0.004527,-0.024098,-0.001568,-0.010525,-0.008056,-0.004066,-0.003650,-0.012772,0.004506,-0.003320,...,-0.011128,-0.010187,-0.005503,-0.002600,0.000089,-0.003188,-0.018486,-0.019023,-0.001643,0.000000
30/12/2020,0.039099,-0.002646,-0.002303,0.010522,0.009259,0.001745,0.004819,0.000000,0.005840,0.005659,...,0.004663,-0.003997,-0.002237,0.004380,-0.002224,0.002430,0.006512,0.010099,0.001462,-0.002275
31/12/2020,0.002172,0.005565,0.012129,0.008361,0.007367,0.002896,0.010301,0.002535,-0.027694,0.021117,...,0.004981,0.001000,0.005948,0.014620,0.010367,0.000435,-0.006047,0.017838,-0.001657,-0.001138


In [79]:
cluster_returns = pd.DataFrame(index=df_cleaned.index[:-1], columns=np.arange(number_of_clusters), data=np.zeros((df_cleaned.shape[0] - 1, number_of_clusters)))

for ticker in df_cleaned.columns:
    cluster_returns[int(df_cleaned[ticker]['Cluster'])] = cluster_returns[int(df_cleaned[ticker]['Cluster'])] + constituent_weights[ticker]['Weight'] * df_cleaned[ticker][:-1]